In [1]:
def cheapest_link_algorithm(cost_matrix):
    # Number of cities
    n = len(cost_matrix)
    # Create a list to keep track of visited cities
    visited = [False] * n
    # Start at the first city
    visited[0] = True
    tour = [0]
    total_cost = 0

    while len(tour) < n:
        min_cost = float('inf')
        for city in tour:
            for next_city in range(n):
                if not visited[next_city] and 0 < cost_matrix[city][next_city] < min_cost:
                    min_cost = cost_matrix[city][next_city]
                    current_city = city
                    next_city_to_visit = next_city
        tour.append(next_city_to_visit)
        visited[next_city_to_visit] = True
        total_cost += min_cost
        print(f"Added edge from {current_city} to {next_city_to_visit} with cost {min_cost}")

    # Add cost to return to the starting city
    total_cost += cost_matrix[tour[-1]][tour[0]]
    print(f"Added return edge from {tour[-1]} to {tour[0]} with cost {cost_matrix[tour[-1]][tour[0]]}")
    
    return tour, total_cost

# Example usage
cost_matrix = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]

tour, total_cost = cheapest_link_algorithm(cost_matrix)
print("Tour:", tour)
print("Total Cost:", total_cost)


Added edge from 0 to 1 with cost 10
Added edge from 0 to 2 with cost 15
Added edge from 0 to 3 with cost 20
Added return edge from 3 to 0 with cost 20
Tour: [0, 1, 2, 3]
Total Cost: 65


In [3]:
import numpy as np

def calculate_probability(distance_matrix, pheromone_matrix, beta, alpha, current_city, unvisited):
    pheromone = np.power(pheromone_matrix[current_city][unvisited], alpha)
    heuristic = np.power(1 / distance_matrix[current_city][unvisited], beta)
    return pheromone * heuristic / np.sum(pheromone * heuristic)

def update_pheromone(pheromone_matrix, ants_tours, distance_matrix, rho, Q):
    # Subtract pheromone by evaporation rate
    pheromone_matrix *= (1 - rho)
    # Add new pheromone
    for tour, distance in ants_tours:
        for i in range(len(tour) - 1):
            pheromone_matrix[tour[i]][tour[i + 1]] += Q / distance
            pheromone_matrix[tour[i + 1]][tour[i]] += Q / distance  # symmetric TSP
    return pheromone_matrix

def aco_tsp(distance_matrix, n_ants, n_iterations, alpha, beta, rho, Q):
    n_cities = distance_matrix.shape[0]
    # Initialize pheromone levels
    pheromone_matrix = np.ones((n_cities, n_cities)) / n_cities
    best_tour = None
    best_distance = float('inf')
    
    for iteration in range(n_iterations):
        ants_tours = []
        for ant in range(n_ants):
            tour = [np.random.randint(n_cities)]
            while len(tour) < n_cities:
                current_city = tour[-1]
                unvisited = list(set(range(n_cities)) - set(tour))
                prob = calculate_probability(distance_matrix, pheromone_matrix, beta, alpha, current_city, unvisited)
                next_city = np.random.choice(unvisited, p=prob)
                tour.append(next_city)
            tour.append(tour[0])  # Return to start
            
            # Calculate the distance of the tour
            distance = sum([distance_matrix[tour[i]][tour[i+1]] for i in range(len(tour) - 1)])
            
            if distance < best_distance:
                best_tour = tour
                best_distance = distance
                
            ants_tours.append((tour, distance))
        
        # Update pheromones
        pheromone_matrix = update_pheromone(pheromone_matrix, ants_tours, distance_matrix, rho, Q)
    
    return best_tour, best_distance

# Example usage
distance_matrix = np.array([
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
])

n_ants = 10
n_iterations = 100
alpha = 1.0  # Pheromone importance
beta = 2.0   # Distance importance
rho = 0.5    # Pheromone evaporation rate
Q = 10.0     # Pheromone deposit factor

best_tour, best_distance = aco_tsp(distance_matrix, n_ants, n_iterations, alpha, beta, rho, Q)
print("Best Tour:", best_tour)
print("Best Distance:", best_distance)


Best Tour: [2, 0, 1, 3, 2]
Best Distance: 80


In [3]:
import numpy as np
from math import radians, cos, sin, asin, sqrt

# Haversine formula to calculate the great-circle distance between two points
def haversine(lon1, lat1, lon2, lat2):
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Cheapest Link Algorithm
def cheapest_link_algorithm(distance_matrix):
    n = len(distance_matrix)
    tour = [0]
    total_cost = 0

    while len(tour) < n:
        last_city = tour[-1]
        next_city = np.argmin([distance_matrix[last_city][j] if j not in tour and distance_matrix[last_city][j] > 0 else np.inf for j in range(n)])
        tour.append(next_city)
        total_cost += distance_matrix[last_city][next_city]
    
    # Complete the tour by returning to the starting city
    total_cost += distance_matrix[tour[-1]][tour[0]]
    tour.append(tour[0])
    return tour, total_cost

# ACO Algorithm
def aco_tsp(distance_matrix, n_ants, n_iterations, alpha, beta, rho, Q):
    n_cities = distance_matrix.shape[0]
    pheromone_matrix = np.ones((n_cities, n_cities)) / n_cities
    best_tour = None
    best_distance = float('inf')

    for _ in range(n_iterations):
        ants_tours = []
        for _ in range(n_ants):
            tour = [np.random.randint(n_cities)]
            while len(tour) < n_cities:
                current_city = tour[-1]
                unvisited = list(set(range(n_cities)) - set(tour))
                prob = calculate_probability(distance_matrix, pheromone_matrix, beta, alpha, current_city, unvisited)
                next_city = np.random.choice(unvisited, p=prob)
                tour.append(next_city)
            tour.append(tour[0])  # Return to start
            distance = sum([distance_matrix[tour[i]][tour[i+1]] for i in range(n_cities)])
            if distance < best_distance:
                best_tour = tour
                best_distance = distance
            ants_tours.append((tour, distance))
        pheromone_matrix = update_pheromone(pheromone_matrix, ants_tours, distance_matrix, rho, Q)

    return best_tour, best_distance

def calculate_probability(distance_matrix, pheromone_matrix, beta, alpha, current_city, unvisited):
    pheromone = np.power(pheromone_matrix[current_city][unvisited], alpha)
    heuristic = np.power(1 / distance_matrix[current_city][unvisited], beta)
    return pheromone * heuristic / np.sum(pheromone * heuristic)

def update_pheromone(pheromone_matrix, ants_tours, distance_matrix, rho, Q):
    pheromone_matrix *= (1 - rho)
    for tour, distance in ants_tours:
        for i in range(len(tour) - 1):
            pheromone_matrix[tour[i]][tour[i + 1]] += Q / distance
            pheromone_matrix[tour[i + 1]][tour[i]] += Q / distance  # symmetric TSP
    return pheromone_matrix

# Coordinates of 10 US cities
cities = [
    (37.7749, -122.4194),  # San Francisco
    (34.0522, -118.2437),  # Los Angeles
    (41.8781, -87.6298),   # Chicago
    (40.7128, -74.0060),   # New York
    (29.7604, -95.3698),   # Houston
    (33.4484, -112.0740),  # Phoenix
    (39.9526, -75.1652),   # Philadelphia
    (29.4241, -98.4936),   # San Antonio
    (32.7157, -117.1611),  # San Diego
    (32.7767, -96.7970)    # Dallas
]

# Generate the distance matrix using the Haversine formula
n_cities = len(cities)
distance_matrix = np.zeros((n_cities, n_cities))
for i in range(n_cities):
    for j in range(n_cities):
        if i != j:
            distance_matrix[i][j] = haversine(cities[i][1], cities[i][0], cities[j][1], cities[j][0])

# ACO Parameters
n_ants = 20
n_iterations = 100
alpha = 1.0  # Pheromone importance
beta = 5.0   # Distance importance
rho = 0.1    # Pheromone evaporation rate
Q = 100.0    # Pheromone deposit factor

# Run Cheapest Link Algorithm
cheapest_tour, cheapest_cost = cheapest_link_algorithm(distance_matrix)
print("Cheapest Link Algorithm:")
print("Tour:", cheapest_tour)
print("Cost:", cheapest_cost)

# Run ACO
aco_tour, aco_cost = aco_tsp(distance_matrix, n_ants, n_iterations, alpha, beta, rho, Q)
print("\nACO Algorithm:")
print("Tour:", aco_tour)
print("Cost:", aco_cost)


Cheapest Link Algorithm:
Tour: [0, 1, 8, 5, 7, 4, 9, 2, 6, 3, 0]
Cost: 9869.772450036977

ACO Algorithm:
Tour: [1, 8, 5, 7, 4, 9, 6, 3, 2, 0, 1]
Cost: 9594.912513400712
